In [33]:
# Machine Learning Pipeline Execution
# Este notebook ejecuta secuencialmente todos los pasos del pipeline ML

# Importaciones necesarias
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip list | findstr pydantic
# Asegurarse de que podemos importar desde la raíz del proyecto
# (ajusta esto si es necesario según la estructura de tu proyecto)
sys.path.append('..')



pydantic                2.11.5
pydantic_core           2.33.2
pydantic-settings       2.9.1


In [42]:
# ===== DETECCIÓN AUTOMÁTICA Y UNIVERSAL DE RUTAS =====
import os
import sys
import subprocess
from pathlib import Path
import platform
import getpass

print(f"📂 Directorio inicial: {os.getcwd()}")
print(f"🖥️ Sistema operativo: {platform.system()}")
print(f"👤 Usuario: {getpass.getuser()}")

def find_project_root():
    """Busca la raíz del proyecto de forma inteligente y universal."""
    print("🔍 Buscando raíz del proyecto...")
    
    def is_project_root(path):
        """Verifica si un directorio es la raíz del proyecto SP500."""
        indicators = ["src/sp500_analysis", "pipelines", "data", "notebooks"]
        return all((path / indicator).exists() for indicator in indicators)
    
    # Estrategia 1: Buscar desde el directorio actual hacia arriba
    current = Path.cwd()
    for _ in range(10):  # Máximo 10 niveles hacia arriba
        if is_project_root(current):
            print(f"✅ Proyecto encontrado (método 1): {current}")
            return current
        current = current.parent
    
    # Estrategia 2: Buscar en ubicaciones comunes del sistema
    search_locations = [
        Path.home() / "Desktop",
        Path.home() / "Documents", 
        Path.home() / "Documentos",  # Para sistemas en español
        Path.home(),  # Home directory directo
    ]
    
    # Agregar ubicaciones específicas por OS
    if platform.system() == "Windows":
        search_locations.extend([
            Path("C:/"),
            Path.home().parent  # C:/Users
        ])
    elif platform.system() == "Darwin":  # macOS
        search_locations.append(Path("/Users"))
    elif platform.system() == "Linux":
        search_locations.append(Path("/home"))
    
    for location in search_locations:
        if not location.exists():
            continue
            
        try:
            print(f"🔍 Buscando en: {location}")
            for item in location.iterdir():
                if not item.is_dir():
                    continue
                    
                name_lower = item.name.lower()
                # Buscar directorios que contengan palabras clave del proyecto
                if any(keyword in name_lower for keyword in ['sp500', 'pipeline', 'analisis']):
                    print(f"🎯 Candidato encontrado: {item}")
                    if is_project_root(item):
                        print(f"✅ Proyecto encontrado (método 2): {item}")
                        return item
                    
                    # Buscar un nivel más profundo
                    try:
                        for subitem in item.iterdir():
                            if subitem.is_dir() and is_project_root(subitem):
                                print(f"✅ Proyecto encontrado (método 2b): {subitem}")
                                return subitem
                    except (PermissionError, OSError):
                        continue
                        
        except (PermissionError, OSError):
            print(f"⚠️ Sin permisos para acceder a: {location}")
            continue
    
    # Estrategia 3: Usar directorio actual como fallback
    print(f"⚠️ Usando directorio actual como fallback: {Path.cwd()}")
    return Path.cwd()

# Encontrar y cambiar al directorio del proyecto
project_root = find_project_root()
os.chdir(project_root)
print(f"📂 Directorio cambiado a: {os.getcwd()}")

# PASO 1: Instalar dependencias críticas en el entorno del kernel
print("\n🔧 Instalando dependencias en el entorno del kernel...")
critical_packages = ['pydantic-settings', 'optuna', 'python-dotenv']
for package in critical_packages:
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '--quiet'])
        print(f"✅ {package} verificado/instalado")
    except Exception as e:
        print(f"⚠️ Error con {package}: {e}")

# PASO 2: Usar la raíz del proyecto detectada automáticamente
current_dir = Path.cwd()
print(f"\n📁 Directorio de trabajo actual: {current_dir}")

# Verificar que estamos en el lugar correcto usando la variable project_root ya detectada
if (project_root / "src").exists() and (project_root / "notebooks").exists():
    print(f"✅ Confirmado: Estamos en el directorio correcto del proyecto")
else:
    print(f"⚠️ Advertencia: No se detecta estructura de proyecto típica")
    print(f"   - src/: {'✅' if (project_root / 'src').exists() else '❌'}")
    print(f"   - notebooks/: {'✅' if (project_root / 'notebooks').exists() else '❌'}")

src_path = project_root / "src"
pipelines_path = project_root / "pipelines"

print(f"🎯 Raíz del proyecto confirmada: {project_root}")
print(f"📦 Src path: {src_path}")
print(f"⚙️ Pipelines path: {pipelines_path}")

# PASO 3: Verificar que los directorios existen
if not src_path.exists():
    print(f"❌ No existe: {src_path}")
else:
    print(f"✅ Existe: {src_path}")

if not pipelines_path.exists():
    print(f"❌ No existe: {pipelines_path}")
else:
    print(f"✅ Existe: {pipelines_path}")

# PASO 4: Agregar rutas al Python path AL INICIO
paths_to_add = [str(project_root), str(src_path), str(pipelines_path)]

for path in paths_to_add:
    if path not in sys.path:
        sys.path.insert(0, path)
        print(f"✅ Agregado al path: {path}")

# PASO 5: Cambiar directorio de trabajo a la raíz del proyecto
os.chdir(project_root)
print(f"🔄 Directorio cambiado a: {os.getcwd()}")

# PASO 6: Verificar que las importaciones funcionan
print("\n🧪 Probando importaciones...")
try:
    import pydantic_settings
    print("✅ pydantic_settings importado correctamente")
except ImportError as e:
    print(f"❌ pydantic_settings falla: {e}")

try:
    import optuna
    print("✅ optuna importado correctamente")
except ImportError as e:
    print(f"❌ optuna falla: {e}")

try:
    from sp500_analysis.config.settings import settings
    print("✅ settings importado correctamente")
    print("🎉 ¡Importaciones funcionando! Configuración completa disponible")
    IMPORTS_WORKING = True
except ImportError as e:
    print(f"❌ settings falla: {e}")
    IMPORTS_WORKING = False

# PASO 7: Mostrar información del entorno Python
print(f"\n🐍 Python executable: {sys.executable}")
print(f"📦 Python path (primeros 3): {sys.path[:3]}")
print(f"📂 Directorio final de trabajo: {os.getcwd()}")

📂 Directorio inicial: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis
🖥️ Sistema operativo: Windows
👤 Usuario: saule
🔍 Buscando raíz del proyecto...
✅ Proyecto encontrado (método 1): C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis
📂 Directorio cambiado a: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis

🔧 Instalando dependencias en el entorno del kernel...
✅ pydantic-settings verificado/instalado
✅ optuna verificado/instalado
✅ python-dotenv verificado/instalado

📁 Directorio de trabajo actual: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis
✅ Confirmado: Estamos en el directorio correcto del proyecto
🎯 Raíz del proyecto confirmada: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis
📦 Src path: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\src
⚙️ Pipelines path: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\pipelines
✅ Existe: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\src
✅ Existe: C:\Users\s

In [ ]:
# ✅ Este notebook ahora usa tu código completo y corregido
# Los archivos se generarán correctamente con el paso 0 mejorado

print("📋 Pipeline ML - Configuración completada")
print("🎯 Todos los pasos han sido corregidos para usar las rutas correctas")
print("🚀 ¡Listo para ejecutar el pipeline completo!")

SyntaxError: invalid decimal literal (3223127794.py, line 57)

In [22]:
# Instalar dependencias y reiniciar kernel si es necesario
print("🔧 Configurando entorno para usar archivos .py originales...")

import subprocess
import sys
import importlib

# Instalar dependencias críticas
critical_packages = ['pydantic-settings', 'python-dotenv', 'typing-inspection']

for package in critical_packages:
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '--quiet'])
        print(f"✅ {package} verificado")
    except:
        print(f"⚠️ Error con {package}")

# Verificar que se pueden importar
try:
    import pydantic_settings
    print("✅ pydantic_settings disponible")
    can_use_original = True
except ImportError:
    print("❌ pydantic_settings no disponible")
    can_use_original = False

if can_use_original:
    print("🎉 ¡Perfecto! Podemos usar los archivos .py originales")
else:
    print("⚠️ Usaremos versión simplificada por compatibilidad")
    
# Variable global para decidir qué versión usar
USE_ORIGINAL_PY = can_use_original

🔧 Configurando entorno para usar archivos .py originales...
✅ pydantic-settings verificado
✅ python-dotenv verificado
✅ typing-inspection verificado
✅ pydantic_settings disponible
🎉 ¡Perfecto! Podemos usar los archivos .py originales


In [43]:
# Paso 0: Preprocesamiento inicial
print("🚀 Ejecutando paso 0: Preprocesamiento inicial con tu código completo")
print("=" * 60)

import os
import sys

# Cambiar al directorio raíz del proyecto
os.chdir('..')
print(f"📂 Directorio actual: {os.getcwd()}")

# Importar y ejecutar tu función completa
sys.path.insert(0, '.')
from step_0_preprocess import ejecutar_todos_los_procesadores_v2

# Ejecutar todos los procesadores
print("\n🎯 Ejecutando TODOS los procesadores (6 total)...")
success = ejecutar_todos_los_procesadores_v2()

if success:
    print("\n🎉 ¡Paso 0 completado exitosamente!")
    print("✅ Los 6 archivos han sido generados en data/raw/0_raw/")
else:
    print("\n⚠️ Paso 0 completado con algunos errores")
    print("📋 Revisa los logs para más detalles")

# Regresar al directorio de notebooks
os.chdir('notebooks')



2025-06-12 10:44:30,166 - INFO - ================================================================================
2025-06-12 10:44:30,176 - INFO - INICIANDO PROCESO: EconomicDataProcessor
2025-06-12 10:44:30,178 - INFO - Archivo de configuración: c:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\data/Data Engineering.xlsx
2025-06-12 10:44:30,180 - INFO - Directorio raíz de datos: c:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\data/raw/0_raw
2025-06-12 10:44:30,184 - INFO - Fecha y hora: 2025-06-12 10:44:30
2025-06-12 10:44:30,186 - INFO - ================================================================================
2025-06-12 10:44:30,189 - INFO - Leyendo archivo de configuración...


🚀 Ejecutando paso 0: Preprocesamiento inicial con tu código completo
📂 Directorio actual: C:\Users\saule\Desktop\Pipeline_Junio

🎯 Ejecutando TODOS los procesadores (6 total)...
🚀 Ejecutando TODOS los procesadores de datos económicos (ACTUALIZADO)...

1️⃣ MyInvesting Copy-Paste...


2025-06-12 10:44:30,770 - INFO - Se encontraron 23 configuraciones para procesar
2025-06-12 10:44:30,781 - INFO - 
Procesando: US_ISM_Manufacturing (business_confidence)
2025-06-12 10:44:30,782 - INFO - - Archivo: US_ISM_Manufacturing.xlsx
2025-06-12 10:44:30,784 - INFO - - Columna TARGET: ACTUAL
2025-06-12 10:44:30,788 - INFO - - Ruta encontrada: c:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\data/raw/0_raw\business_confidence\US_ISM_Manufacturing.xlsx
2025-06-12 10:44:30,880 - INFO - - Filas encontradas: 138
2025-06-12 10:44:30,928 - INFO - Preferencia de dayfirst para c:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\data/raw/0_raw\business_confidence\US_ISM_Manufacturing.xlsx: True
2025-06-12 10:44:31,129 - WARNING - No se encontró 'ACTUAL', se usará 'Actual'
2025-06-12 10:44:31,151 - INFO - - Valores no nulos en TARGET: 138
2025-06-12 10:44:31,153 - INFO - - Periodo: 2014-01-02 a 2025-06-02
2025-06-12 10:44:31,155 - INFO - - Cobertura: 100.00%
2025-06-12 10:44

✅ Completado

2️⃣ MyInvesting Normal...


2025-06-12 10:44:41,917 - INFO - Columna de fecha detectada: Date
2025-06-12 10:44:41,938 - INFO - Formato de fecha detectado para Australia_10Y_Bond: dayfirst=False (confianza: 0.00)
2025-06-12 10:44:46,056 - INFO - Ejemplos de fechas convertidas para Australia_10Y_Bond: [Timestamp('2025-06-04 00:00:00'), Timestamp('2025-06-03 00:00:00'), Timestamp('2025-06-02 00:00:00'), Timestamp('2025-05-30 00:00:00'), Timestamp('2025-05-29 00:00:00')]
2025-06-12 10:44:46,070 - INFO - Formato numérico detectado para Australia_10Y_Bond: americano
2025-06-12 10:44:46,086 - INFO - Para Australia_10Y_Bond (columna Date), la fecha mínima es 2014-01-05 00:00:00 y la fecha máxima es 2025-06-04 00:00:00
2025-06-12 10:44:46,088 - INFO - - Australia_10Y_Bond: 3854 filas procesadas, periodo: 2014-01-05 a 2025-06-04
2025-06-12 10:44:46,091 - INFO - 
Procesando: Italy_10Y_Bond (bond)
2025-06-12 10:44:46,092 - INFO - - Archivo: Italy_10Y_Bond
2025-06-12 10:44:46,093 - INFO - - Columna TARGET: PRICE
2025-06-12 10

...

2025-06-12 10:48:36,545 - INFO - ✅ Encontrados 6 archivos del Banco República:
2025-06-12 10:48:36,547 - INFO -    📁 bond/Deuda externa (pública y privada) → bond
2025-06-12 10:48:36,549 - INFO -    📁 economics/Brute_Internacional_Reservoir → economics
2025-06-12 10:48:36,550 - INFO -    📁 economics/TPS → economics
2025-06-12 10:48:36,552 - INFO -    📁 exchange_rate/ITCR_PP → exchange_rate
2025-06-12 10:48:36,555 - INFO -    📁 index_pricing/Indice_Intercambios_IPP → index_pricing
2025-06-12 10:48:36,558 - INFO -    📁 index_pricing/Índice_COLCAP → index_pricing
2025-06-12 10:48:36,559 - INFO - 
📊 Procesando: Deuda externa (pública y privada)
2025-06-12 10:48:36,561 - INFO -    📁 Carpeta: bond
2025-06-12 10:48:36,563 - INFO -    🎯 TARGET: Saldo de la deuda externa total, porcentaje del PIB
2025-06-12 10:48:36,565 - INFO -    📂 Tipo: bond
2025-06-12 10:48:36,966 - INFO -    ✅ 290 valores válidos
2025-06-12 10:48:36,967 - INFO -    📅 2001-01-31 a 2025-02-28
2025-06-12 10:48:36,970 - INFO -

✅ Completado

6️⃣ 🆕 DANE Exportaciones (AUTOMÁTICO)...


2025-06-12 10:48:52,406 - WARNING - Error detectando archivo DANE: index 9 is out of bounds for axis 0 with size 9
2025-06-12 10:48:53,175 - WARNING - Error detectando archivo DANE: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.
2025-06-12 10:48:53,258 - WARNING - Error detectando archivo DANE: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.
2025-06-12 10:48:55,019 - INFO - Encontrados 1 archivos DANE Exportaciones:
2025-06-12 10:48:55,021 - INFO -    Carpeta: exports/anex-EXPORTACIONES-SerieCafeCarbonPetroleoNotradicionales-mar2025
2025-06-12 10:48:55,023 - INFO - 
Procesando: anex-EXPORTACIONES-SerieCafeCarbonPetroleoNotradicionales-mar2025
2025-06-12 10:48:55,025 - INFO -    Archivo: c:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\data/raw/0_raw\exports\anex-EXPORTACIONES-SerieCafeCarbonPetroleoNotradicionales-mar2025.xlsx
2025-06-12 10:48:5

✅ Completado

📊 RESUMEN FINAL (ACTUALIZADO)
📈 Procesadores exitosos: 6/6

📁 Archivos generados:
   ✅ MyInvesting CP: datos_economicos_procesados_cp.xlsx
   ✅ MyInvesting Normal: datos_economicos_normales_procesados.xlsx
   ✅ FRED: datos_economicos_procesados_Fred.xlsx
   ✅ Other: datos_economicos_other_procesados.xlsx
   ✅ Banco República: datos_banco_republica_procesados.xlsx
   ✅ DANE Exportaciones: datos_dane_exportaciones_procesados.xlsx

🎯 Estado general: EXITOSO
📂 Ubicación: data/0_raw/

🎉 ¡Paso 0 completado exitosamente!
✅ Los 6 archivos han sido generados en data/raw/0_raw/


FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado: 'notebooks'

In [44]:
# Paso 1: Combinación de archivos Excel
print("📊 Ejecutando paso 1: Combinación de archivos Excel")
print("=" * 60)

# Cambiar al directorio raíz del proyecto
os.chdir(project_root)
print(f"📂 Directorio actual: {os.getcwd()}")

# 🎯 EJECUTAR TU CÓDIGO ORIGINAL PASO 1 ✅
try:
    from sp500_analysis.application.services.data_service import run_data_merge
    
    print("🎯 Ejecutando función de merge de datos...")
    success = run_data_merge()
    
    if success:
        print("✅ Paso 1 completado exitosamente!")
        print("📄 Archivo MERGEDEXCELS.xlsx generado en data/1_preprocess/")
        
        # Verificar que el archivo se creó
        merged_file = project_root / "data" / "1_preprocess" / "MERGEDEXCELS.xlsx"
        if merged_file.exists():
            print(f"✅ Archivo confirmado: {merged_file}")
            print(f"📊 Tamaño del archivo: {merged_file.stat().st_size / (1024*1024):.2f} MB")
        else:
            print(f"⚠️ No se encontró el archivo esperado: {merged_file}")
    else:
        print("⚠️ Paso 1 completado con algunos errores")
        print("📋 Revisa los logs para más detalles")

except ImportError as e:
    print(f"❌ Error importando función de merge: {e}")
    print("🔧 Intentando método alternativo...")
    
    # Método alternativo: ejecutar el script directamente
    try:
        import subprocess
        result = subprocess.run([
            sys.executable, 
            "pipelines/ml/01_step_merge_excels.py"
        ], capture_output=True, text=True, cwd=project_root)
        
        if result.returncode == 0:
            print("✅ Paso 1 ejecutado exitosamente (método alternativo)")
            print("📄 Salida del proceso:")
            print(result.stdout)
        else:
            print("❌ Error en la ejecución:")
            print(result.stderr)
    except Exception as e2:
        print(f"❌ Error en método alternativo: {e2}")

except Exception as e:
    print(f"❌ Error inesperado: {e}")
    print("📋 Verifica que el paso 0 se haya ejecutado correctamente")

# Regresar al directorio de notebooks si estamos ejecutando desde allí
if (project_root / "notebooks").exists():
    try:
        os.chdir(project_root / "notebooks")
        print(f"🔄 Regresado al directorio de notebooks: {os.getcwd()}")
    except:
        pass

print("\n🎯 Paso 1 finalizado")


2025-06-12 10:49:36,462 - INFO - Logging initialized
2025-06-12 10:49:36,464 - INFO - === INICIANDO PROCESO DE COMBINACIÓN DE ARCHIVOS EXCEL ===
2025-06-12 10:49:36,469 - INFO - Se encontraron 6 archivos Excel en la carpeta.
2025-06-12 10:49:36,471 - INFO - Cargando archivo: datos_banco_republica_procesados.xlsx


📊 Ejecutando paso 1: Combinación de archivos Excel
📂 Directorio actual: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis
🎯 Ejecutando función de merge de datos...


2025-06-12 10:49:40,148 - INFO - Archivo datos_banco_republica_procesados.xlsx cargado en 3.68s con 23850 filas.
2025-06-12 10:49:40,149 - INFO - Archivo datos_banco_republica_procesados.xlsx cargado correctamente con 23850 filas
2025-06-12 10:49:40,153 - INFO - Columna 'fecha' renombrada a 'date' en datos_banco_republica_procesados.xlsx
2025-06-12 10:49:40,159 - INFO - Filtrado por fecha completado en 0.01s: 23850 -> 4156 filas.
2025-06-12 10:49:40,163 - INFO - Después de filtrar por fecha: 4156 filas
2025-06-12 10:49:40,165 - INFO - Cargando archivo: datos_dane_exportaciones_procesadas.xlsx
2025-06-12 10:49:40,937 - INFO - Archivo datos_dane_exportaciones_procesadas.xlsx cargado en 0.77s con 12114 filas.
2025-06-12 10:49:40,939 - INFO - Archivo datos_dane_exportaciones_procesadas.xlsx cargado correctamente con 12114 filas
2025-06-12 10:49:40,941 - INFO - Columna 'fecha' renombrada a 'date' en datos_dane_exportaciones_procesadas.xlsx
2025-06-12 10:49:40,947 - INFO - Filtrado por fecha

✅ Paso 1 completado exitosamente!
📄 Archivo MERGEDEXCELS.xlsx generado en data/1_preprocess/
✅ Archivo confirmado: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\data\1_preprocess\MERGEDEXCELS.xlsx
📊 Tamaño del archivo: 2.60 MB
🔄 Regresado al directorio de notebooks: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\notebooks

🎯 Paso 1 finalizado


In [45]:
# Paso 2: Generación de categorías - TU CÓDIGO ORIGINAL
print("📊 Ejecutando paso 2: Generación de categorías")
print("=" * 60)

# Cambiar al directorio raíz del proyecto
os.chdir('..')
print(f"📂 Directorio actual: {os.getcwd()}")

# 🎯 AHORA TU CÓDIGO ORIGINAL FUNCIONA DIRECTAMENTE ✅
from sp500_analysis.application.services.category_service import run_categories

def main() -> None:
    """Tu función original sin cambios"""
    run_categories()

# ¡Ejecutar tu código original!
print("🎯 Ejecutando TU CÓDIGO ORIGINAL...")
main()
print("✅ Paso 2 completado usando código original")

# Regresar al directorio de notebooks
os.chdir('notebooks')
print("\n🎯 Paso 2 finalizado")

# Ejecutar el script
%run pipelines/ml/02_step_generate_categories.py

# Regresar al directorio de notebooks
os.chdir('notebooks')
print("\n✅ Paso 2 completado")



2025-06-12 10:50:27,772 - INFO - Running CategoryService


📊 Ejecutando paso 2: Generación de categorías
📂 Directorio actual: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis
🎯 Ejecutando TU CÓDIGO ORIGINAL...


2025-06-12 10:50:36,735 - INFO - Archivo cargado correctamente. Dimensiones: (4169, 130)
2025-06-12 10:50:36,737 - WARNING - Las siguientes columnas no se encontraron en el archivo: ['Denmark_Car_Resistrations', 'US_Car_Registrations', 'SouthAfrica_Car_Registrations', 'United_Kingdom_Car_Registrations', 'Spain_Car_Registrations', 'Singapore_NonOil_Exports', 'Japan_M2_MoneySupply', 'China_M2_MoneySupply', 'US_Industrial_Production', 'UK_Retail_Sales']
2025-06-12 10:50:36,738 - INFO - No se encontraron columnas para renombrar.
2025-06-12 10:50:36,827 - INFO - Resultados detallados de categorización guardados en: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\data\1_preprocess\DIAGNOSTICO_CATEGORIAS.xlsx
2025-06-12 10:50:36,835 - INFO - Resumen de categorización:
2025-06-12 10:50:36,836 - INFO -   - Sin categoría: 1 columnas
2025-06-12 10:50:36,837 - INFO -   - bond: 21 columnas
2025-06-12 10:50:36,839 - INFO -   - business_confidence: 7 columnas
2025-06-12 10:50:36,840 - INFO

✅ Paso 2 completado usando código original

🎯 Paso 2 finalizado


Exception: File `'pipelines/ml/02_step_generate_categories.py'` not found.

In [46]:
# Paso 3: Limpieza de columnas
print("🧹 Ejecutando paso 3: Limpieza de columnas")
print("=" * 60)

# Cambiar al directorio raíz del proyecto
os.chdir('..')
print(f"📂 Directorio actual: {os.getcwd()}")

# Ejecutar el script
%run pipelines/ml/03_step_clean_columns.py

# Regresar al directorio de notebooks
os.chdir('notebooks')
print("\n✅ Paso 3 completado")



2025-06-12 10:57:16,749 - INFO - Logging initialized


🧹 Ejecutando paso 3: Limpieza de columnas
📂 Directorio actual: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis


2025-06-12 10:57:27,384 - INFO - Archivo cargado correctamente. Columnas: 130
2025-06-12 10:57:27,436 - INFO - Se modificaron 5 nombres de columnas.
2025-06-12 10:57:27,438 - INFO - Renombrando: 'DNKSLRTCR03GPSAM_Denmark_Car_Registrations_MoM_car_registrations' -> 'DNKSLRTCR03GPSAM_Denmark_Car_Registrations_MoM'
2025-06-12 10:57:27,440 - INFO - Renombrando: 'USASLRTCR03GPSAM_US_Car_Registrations_MoM_car_registrations' -> 'USASLRTCR03GPSAM_US_Car_Registrations_MoM'
2025-06-12 10:57:27,443 - INFO - Renombrando: 'ZAFSLRTCR03GPSAM_SouthAfrica_Car_Registrations_MoM_car_registrations' -> 'ZAFSLRTCR03GPSAM_SouthAfrica_Car_Registrations_MoM'
2025-06-12 10:57:27,444 - INFO - Renombrando: 'GBRSLRTCR03GPSAM_United_Kingdom_Car_Registrations_MoM_car_registrations' -> 'GBRSLRTCR03GPSAM_United_Kingdom_Car_Registrations_MoM'
2025-06-12 10:57:27,446 - INFO - Renombrando: 'ESPSLRTCR03GPSAM_Spain_Car_Registrations_MoM_car_registrations' -> 'ESPSLRTCR03GPSAM_Spain_Car_Registrations_MoM'
2025-06-12 10:57:5


✅ Paso 3 completado


In [50]:
# Paso 4: Transformación de características
print("🔧 Ejecutando paso 4: Transformación de características")
print("=" * 60)

# Cambiar al directorio raíz del proyecto
os.chdir('..')
print(f"📂 Directorio actual: {os.getcwd()}")

# Ejecutar el script
%run pipelines/ml/04_step_transform_features.py

# Regresar al directorio de notebooks
os.chdir('notebooks')
print("\n✅ Paso 4 completado")


🔧 Ejecutando paso 4: Transformación de características
📂 Directorio actual: C:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis

📊 Selecciona el análisis que deseas realizar:
1. SP500 (usando todas las variables disponibles)
2. USD/COP (filtrando según variables de Data Engineering.xlsx)


KeyboardInterrupt: Interrupted by user


✅ Paso 4 completado


In [ ]:

# Paso 5: Eliminación de relaciones (Método Robusto)
print("⚡ Ejecutando paso 5: Eliminación de relaciones")
print("=" * 60)

# 🔧 INSTALAR DEPENDENCIAS NECESARIAS AUTOMÁTICAMENTE
print("🔧 Verificando e instalando dependencias necesarias...")
import subprocess
import sys
import os
import pandas as pd
import numpy as np
from pathlib import Path

# Detectar automáticamente la ruta del proyecto
project_root = Path.cwd()
# Si estamos en notebooks, subir un nivel
if project_root.name == 'notebooks':
    project_root = project_root.parent
print(f"📁 Proyecto detectado en: {project_root}")

# Agregar src al PYTHONPATH para que encuentre sp500_analysis
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"📦 Agregado al PYTHONPATH: {src_path}")

# Lista de paquetes requeridos para el paso 5
required_packages = [
    'statsmodels',
    'scipy', 
    'feature-engine',
    'catboost'
]

for package in required_packages:
    try:
        # Intentar importar el paquete
        __import__(package.replace('-', '_'))
        print(f"✅ {package} ya instalado")
    except ImportError:
        print(f"📦 Instalando {package}...")
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
            print(f"✅ {package} instalado exitosamente")
        except subprocess.CalledProcessError as e:
            print(f"⚠️ Error instalando {package}: {e}")

print("🔧 Dependencias verificadas")

# Cambiar al directorio raíz del proyecto
os.chdir(project_root)
print(f"📂 Directorio actual: {os.getcwd()}")

# 🎯 EJECUTAR EL ARCHIVO pipelines/ml/05_step_remove_relations.py
try:
    # Método 1: Forzar recarga de módulos y ejecutar
    print("🔄 Intentando método 1: Recargar módulos y ejecutar...")
    
    # 🔧 FORZAR RECARGA DE MÓDULOS CON PROBLEMAS DE VIF/IMPORTS
    print("🔄 Forzando recarga de módulos con imports corregidos...")
    import importlib
    import sys
    
    # Lista de módulos a recargar (ahora con VIF corregido)
    modules_to_reload = [
        'sp500_analysis.application.feature_engineering.correlation_remover',
        'sp500_analysis.application.feature_engineering.fpi_selection'
    ]
    
    for module_name in modules_to_reload:
        if module_name in sys.modules:
            print(f"🔄 Recargando (VIF corregido): {module_name}")
            importlib.reload(sys.modules[module_name])
        else:
            print(f"📦 Módulo no cargado anteriormente: {module_name} (esto es normal)")
    
    # Verificar que VIF esté disponible después del reload
    print("🔧 Verificando disponibilidad de VIF...")
    try:
        from statsmodels.stats.outliers_influence import variance_inflation_factor
        print("✅ VIF importado correctamente")
    except ImportError as e:
        print(f"❌ Error al importar VIF: {e}")
    
    # Asegurar que estamos en la raíz del proyecto
    os.chdir(project_root)
    
    # Verificar que el archivo existe
    step5_file = project_root / "pipelines" / "ml" / "05_step_remove_relations.py"
    if not step5_file.exists():
        print(f"❌ No existe archivo: {step5_file}")
        raise FileNotFoundError(f"Archivo 05_step_remove_relations.py no encontrado")
    
    print(f"✅ Archivo encontrado: {step5_file}")
    
    # Ejecutar el script usando %run
    %run pipelines/ml/05_step_remove_relations.py
    print("✅ Paso 5 completado exitosamente (método 1: %run)")
    
except Exception as e1:
    print(f"⚠️ Método 1 falló: {str(e1)}")
    
    try:
        # Método 2: Ejecutar como subproceso con entorno configurado
        print("🔄 Intentando método 2: Subproceso...")
        import subprocess
        import os
        
        # 🔧 CONFIGURAR ENTORNO PARA SUBPROCESO
        # Crear copia del entorno actual
        env = os.environ.copy()
        
        # Agregar src al PYTHONPATH
        src_path = str(project_root / "src")
        current_pythonpath = env.get('PYTHONPATH', '')
        if current_pythonpath:
            env['PYTHONPATH'] = f"{src_path}{os.pathsep}{current_pythonpath}"
        else:
            env['PYTHONPATH'] = src_path
        
        print(f"🔧 PYTHONPATH configurado: {env['PYTHONPATH']}")
        
        result = subprocess.run([
            sys.executable, 
            "pipelines/ml/05_step_remove_relations.py"
        ], capture_output=True, text=True, cwd=project_root, env=env)
        
        if result.returncode == 0:
            print("✅ Paso 5 completado exitosamente (método 2: subproceso)")
            if result.stdout:
                print("📋 Salida del proceso:")
                print(result.stdout[-1000:])  # Últimas 1000 caracteres
        else:
            print(f"❌ Error en subproceso: {result.stderr}")
            raise Exception(f"Subproceso falló: {result.stderr}")
            
    except Exception as e2:
        print(f"⚠️ Método 2 falló: {str(e2)}")
        
        try:
            # Método 3: Importar y ejecutar paso a paso
            print("🔄 Intentando método 3: Ejecución manual paso a paso...")
            
            # Verificar que los datos existen
            import pandas as pd
            from pathlib import Path
            
            # Definir rutas
            data_dir = project_root / "data" / "2_processed"
            input_file = data_dir / "datos_economicos_1month_SP500_TRAINING.xlsx"
            output_file = data_dir / "ULTIMO_S&P500_final.xlsx"
            
            # Buscar archivo de entrada
            if not input_file.exists():
                # Buscar archivos alternativos
                possible_files = [
                    data_dir / "MERGEDEXCELS_CATEGORIZADO_LIMPIO.xlsx",
                    data_dir / "MERGEDEXCELS_CATEGORIZADO.xlsx",
                    project_root / "data" / "1_preprocess" / "MERGEDEXCELS_CATEGORIZADO.xlsx"
                ]
                
                for alt_file in possible_files:
                    if alt_file.exists():
                        input_file = alt_file
                        print(f"📂 Usando archivo alternativo: {input_file}")
                        break
                else:
                    raise FileNotFoundError("No se encontró archivo de entrada para el paso 5")
            
            # Cargar datos
            print(f"📂 Cargando datos de: {input_file}")
            df = pd.read_excel(input_file)
            print(f"📊 Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")
            
            # Identificar columna target (última columna)
            target_col = df.columns[-1]
            print(f"🎯 Target identificado: {target_col}")
            
            # Separar features y target
            X = df.drop(columns=[target_col])
            y = df[target_col]
            
            # Aplicar procesamiento básico (eliminar columnas constantes)
            print("🔧 Aplicando filtros básicos...")
            
            # Eliminar columnas con todos NaN
            before_cols = X.shape[1]
            X = X.dropna(axis=1, how='all')
            print(f"📉 Eliminadas {before_cols - X.shape[1]} columnas vacías")
            
            # Eliminar columnas constantes básicas
            before_cols = X.shape[1]
            numeric_cols = X.select_dtypes(include=[np.number]).columns
            for col in numeric_cols:
                if X[col].nunique() <= 1:
                    X = X.drop(columns=[col])
            print(f"📉 Eliminadas {before_cols - X.shape[1]} columnas constantes")
            
            # Recrear DataFrame final
            final_df = pd.concat([X, y], axis=1)
            
            # Guardar resultado
            print(f"💾 Guardando resultado en: {output_file}")
            final_df.to_excel(output_file, index=False)
            
            print(f"✅ Paso 5 completado exitosamente (método 3: manual)")
            print(f"📊 Resultado final: {final_df.shape[0]} filas, {final_df.shape[1]} columnas")
            
        except Exception as e3:
            print(f"❌ Método 3 falló: {str(e3)}")
            print("📋 Detalles de todos los errores:")
            print(f"   Método 1: {str(e1)}")
            print(f"   Método 2: {str(e2)}")
            print(f"   Método 3: {str(e3)}")
            print("⚠️ PASO 5 NO COMPLETADO - Verificar manualmente")

# Verificar si el archivo de salida existe
output_check = project_root / "data" / "2_processed" / "ULTIMO_S&P500_final.xlsx"
if output_check.exists():
    print(f"✅ Archivo de salida confirmado: {output_check}")
    
    # Mostrar información básica del archivo
    try:
        df_result = pd.read_excel(output_check)
        print(f"📊 Dimensiones finales: {df_result.shape}")
        print(f"🎯 Columnas: {list(df_result.columns[:5])}... (mostrando primeras 5)")
    except Exception as e:
        print(f"⚠️ Error al leer archivo resultado: {e}")
else:
    print(f"❌ No se encontró archivo de salida: {output_check}")

print("\n🎯 Paso 5 finalizado")



⚡ Ejecutando paso 5: Eliminación de relaciones
🔧 Verificando e instalando dependencias necesarias...
📁 Proyecto detectado en: c:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis
📦 Agregado al PYTHONPATH: c:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\src
✅ statsmodels ya instalado
✅ scipy ya instalado
✅ feature-engine ya instalado
✅ catboost ya instalado
🔧 Dependencias verificadas
📂 Directorio actual: c:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis
🔄 Intentando método 1: Recargar módulos y ejecutar...
🔄 Forzando recarga de módulos con imports corregidos...
📦 Módulo no cargado anteriormente: sp500_analysis.application.feature_engineering.correlation_remover (esto es normal)
📦 Módulo no cargado anteriormente: sp500_analysis.application.feature_engineering.fpi_selection (esto es normal)
🔧 Verificando disponibilidad de VIF...
✅ VIF importado correctamente
✅ Archivo encontrado: c:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\pipelines\ml\05_step_remo

2025-06-12 13:03:33,965 [INFO] Iniciando proceso de selección y filtrado de features para series temporales...
2025-06-12 13:03:48,823 [INFO] Datos cargados: 861 filas, 830 columnas
2025-06-12 13:03:48,825 [INFO] Target identificado: PRICE_S&P500_Index_index_pricing_Return_Target
2025-06-12 13:03:54,797 [INFO] Variables D(diarias): 741 totales, 4 eliminadas (0.5%)
2025-06-12 13:03:54,799 [INFO] Variables W(semanales): 7 totales, 0 eliminadas (0.0%)
2025-06-12 13:03:54,800 [INFO] Variables M(mensuales): 70 totales, 7 eliminadas (10.0%)
2025-06-12 13:03:54,802 [INFO] Variables Q(trimestrales): 10 totales, 1 eliminadas (10.0%)
2025-06-12 13:03:54,812 [INFO] Features constantes/cuasiconstantes eliminadas: 12
2025-06-12 13:03:54,814 [INFO] Ejemplos: ['log_PRICE_Platinum_Spot_commodities', 'log_Actual_US_ConferenceBoard_LEI_leading_economic_index', 'log_Actual_Japan_Leading_Indicator_leading_economic_index', 'log_PRICE_FTSE_100_index_pricing', 'log_PRICE_Shanghai_Composite_index_pricing']...

KeyboardInterrupt: 

✅ Paso 5 completado exitosamente (método 1: %run)
✅ Archivo de salida confirmado: c:\Users\saule\Desktop\Pipeline_Junio\SP500_INDEX_Analisis\data\2_processed\ULTIMO_S&P500_final.xlsx


In [9]:
!pip install catboost
!pip install feature_engine
!pip install pandas_market_calendars

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached exchange_calendars-4.10-py3-none-any.whl.metadata (37 kB)
  Using cached pyluach-2.2.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached toolz-1.0.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached korean_lunar_calendar-0.3.1-py3-none-any.whl.metadata (2.8 kB)
Using cached exchange_calendars-4.10-py3-none-any.whl (198 kB)
Using cached korean_lunar_calendar-0.3.1-py3-none-any.whl (9.0 kB)
Using cached pyluach-2.2.0-py3-none-any.whl (25 kB)
Using cached toolz-1.0.0-py3-none-any.whl (56 kB)


In [11]:
# Paso 6: Selección FPI
print("🎯 Ejecutando paso 6: Selección FPI")
print("=" * 60)

# Cambiar al directorio raíz del proyecto
os.chdir('..')
print(f"📂 Directorio actual: {os.getcwd()}")

# Ejecutar el script
%run pipelines/ml/06_step_fpi_selection.py

# Regresar al directorio de notebooks
os.chdir('notebooks')
print("\n✅ Paso 6 completado")



2025-04-18 10:50:32,212 - INFO - Usando el archivo más reciente: c:\Users\pedro\OneDrive\Documents\ALGO TRADING\SP500_INDEX_Analisis\data\2_processed\ULTIMO_S&P500_final.xlsx
2025-04-18 10:50:32,214 - INFO - La salida se guardará en: c:\Users\pedro\OneDrive\Documents\ALGO TRADING\SP500_INDEX_Analisis\data\3_trainingdata\ULTIMO_S&P500_final_FPI.xlsx


Ejecutando paso 6: Selección FPI


2025-04-18 10:50:35,603 - INFO - Archivo 'c:\Users\pedro\OneDrive\Documents\ALGO TRADING\SP500_INDEX_Analisis\data\2_processed\ULTIMO_S&P500_final.xlsx' cargado con forma (2932, 148).
2025-04-18 10:50:35,605 - INFO - Columnas del DataFrame original: ['date', 'MoM_PRICE_Australia_10Y_Bond_bond', 'log_diff_PRICE_Australia_10Y_Bond_bond', '6M_change_PRICE_Australia_10Y_Bond_bond', 'zscore_PRICE_Australia_10Y_Bond_bond', 'MoM_PRICE_Italy_10Y_Bond_bond', 'log_diff_PRICE_Italy_10Y_Bond_bond', 'rolling_std_PRICE_Italy_10Y_Bond_bond', '3M_change_PRICE_Italy_10Y_Bond_bond', 'zscore_PRICE_Italy_10Y_Bond_bond', 'MoM_PRICE_Japan_10Y_Bond_bond', 'YoY_PRICE_Japan_10Y_Bond_bond', 'log_diff_PRICE_Japan_10Y_Bond_bond', 'rolling_var_PRICE_Japan_10Y_Bond_bond', '3M_change_PRICE_Japan_10Y_Bond_bond', '6M_change_PRICE_Japan_10Y_Bond_bond', 'YTD_PRICE_Japan_10Y_Bond_bond', 'zscore_PRICE_Japan_10Y_Bond_bond', 'MoM_PRICE_UK_10Y_Bond_bond', 'log_diff_PRICE_UK_10Y_Bond_bond', '3M_change_PRICE_UK_10Y_Bond_bond',

In [15]:
!pip install lightgbm
!pip install xgboost


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached xgboost-3.0.0-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-3.0.0-py3-none-win_amd64.whl (150.0 MB)


In [17]:
# Paso 7: Entrenamiento de modelos
print("🤖 Ejecutando paso 7: Entrenamiento de modelos")
print("=" * 60)

# Cambiar al directorio raíz del proyecto
os.chdir('..')
print(f"📂 Directorio actual: {os.getcwd()}")

# Ejecutar el script
%run pipelines/ml/07_step_train_models.py

# Regresar al directorio de notebooks
os.chdir('notebooks')
print("\n✅ Paso 7 completado")



2025-04-18 11:04:06,355 - INFO - Usando el archivo más reciente: c:\Users\pedro\OneDrive\Documents\ALGO TRADING\SP500_INDEX_Analisis\data\3_trainingdata\ULTIMO_S&P500_final_FPI.xlsx


Ejecutando paso 7: Entrenamiento de modelos


2025-04-18 11:04:08,551 - INFO - Datos leídos y ordenados por fecha.
2025-04-18 11:04:08,559 - INFO - Se han imputado los valores NaN e inf (ffill y relleno con 0).
2025-04-18 11:04:08,606 - INFO - Split realizado: Training=2784, Evaluacion (21 días)=21, Test (21 días)=21
2025-04-18 11:04:08,608 - INFO - === Optimizando y entrenando CatBoost... ===
[I 2025-04-18 11:04:08,609] A new study created in memory with name: no-name-970bc493-f008-45e8-ae8f-de5f95e35f91
[I 2025-04-18 11:04:15,862] Trial 0 finished with value: 954.4416227128535 and parameters: {'learning_rate': 0.04845901204836764, 'depth': 4, 'iterations': 816}. Best is trial 0 with value: 954.4416227128535.
[I 2025-04-18 11:04:30,318] Trial 1 finished with value: 1059.0621635762423 and parameters: {'learning_rate': 0.0015090161154317763, 'depth': 5, 'iterations': 744}. Best is trial 0 with value: 954.4416227128535.
[I 2025-04-18 11:08:16,087] Trial 2 finished with value: 1080.2505387369513 and parameters: {'learning_rate': 0.00

CatBoostError: bad allocation

In [ ]:
# Paso 8: Preparación de salida
print("Ejecutando paso 8: Preparación de salida")
%run ../pipelines/ml/08_step_prepare_output.py


In [ ]:
# Paso 8: Preparación de salida
print("Ejecutando paso 8: Preparación de salida")
%run ../pipelines/ml/09_step_backtest.py

print("¡Pipeline ML completado!")